# Demo de algoritmos para almacenar passwords

In [1]:
password = b'cbda'

In [2]:
ALPHABET = 'abcd'

## Probando las funciones hash clásicas: MD5, SHA1 y SHA256

In [3]:
import hashlib

In [4]:
md5_digest = hashlib.md5(password).hexdigest()
print(md5_digest)

4abf4d24c623f9a3fa9ca35f68b9ce17


In [5]:
sha1_digest = hashlib.sha1(password).hexdigest()
print(sha1_digest)

270a5f78075011a3f9d3861b20e9dc5161df8851


In [6]:
sha256_digest = hashlib.sha256(password).hexdigest()
print(sha256_digest)

b6d838d324733bba724b52e721620abcbde75183e449a42b3f4edfa974a3222b


Ahora, una función que me permita generar todas las passwords posibles (del largo del alfabeto) para poder hacer fuerza bruta, y una segunda que me permita hacer la fuerza bruta para cada algoritmo:

In [7]:
import itertools

def get_bruteforce_string_list():
    prod = itertools.product(ALPHABET, repeat = len(ALPHABET))
    return (("".join(map(str, a)).encode('utf-8') for a in prod))
def bruteforce_hash(hash_object, password_hash):
    for attempt in get_bruteforce_string_list():
        h = hashlib.new(hash_object)
        h.update(attempt)
        if h.hexdigest() == password_hash:
            print('PASSWORD FOUND: %s' % attempt)
            break
def bruteforce_plaintext():
    for attempt in get_bruteforce_string_list():
        if attempt == password:
            print('PASSWORD FOUND: %s' % attempt)
            break

#### Fuerza bruta para texto plano

In [8]:
timeit_plaintext = %timeit -n1 -r1 -q -o bruteforce_plaintext()

PASSWORD FOUND: b'cbda'


#### Fuerza bruta para MD5

In [9]:
timeit_md5 = %timeit -n1 -r1 -q -o bruteforce_hash('md5', md5_digest)

PASSWORD FOUND: b'cbda'


#### Fuerza bruta para SHA1

In [10]:
timeit_sha1 = %timeit -n1 -r1 -q -o bruteforce_hash('sha1', sha1_digest)

PASSWORD FOUND: b'cbda'


#### Fuerza bruta para SHA256

In [11]:
timeit_sha256 = %timeit -n1 -r1 -q -o bruteforce_hash('sha256', sha256_digest)

PASSWORD FOUND: b'cbda'


## Ahora con una función KDF: BCrypt

In [12]:
import bcrypt

Generación de una sal. A la función `gensalt` se le puede dar un parámetro de costo. Por defecto es 12. Ver <https://en.wikipedia.org/wiki/Bcrypt>

In [13]:
salt = bcrypt.gensalt()
print(salt)

b'$2b$12$Etk7tLlUKqXIKzTLeLfwtO'


Generación del KDF o "digest" respectivo:

In [14]:
kdf_bcrypt = bcrypt.hashpw(password, salt)
print(kdf_bcrypt)

b'$2b$12$Etk7tLlUKqXIKzTLeLfwtOUWsyT0gMm1KUPHFH2na1mx.lOE1WkYi'


#### Fuerza bruta para BCrypt

In [15]:
def bruteforce_bcrypt():
    for attempt in get_bruteforce_string_list():
        if bcrypt.hashpw(attempt, kdf_bcrypt) == kdf_bcrypt:
            print('PASSWORD FOUND: %s' % attempt)
            break
timeit_bcrypt = %timeit -n1 -r1 -q -o bruteforce_bcrypt()

PASSWORD FOUND: b'cbda'


## Resultados

In [16]:
from ipy_table import make_table, apply_theme
results_data = [
    ['Tipo', 'Tiempo (s)'],
    ['Texto plano', timeit_plaintext.best],
    ['MD5', timeit_md5.best],
    ['SHA1', timeit_sha1.best],
    ['SHA256', timeit_sha256.best],
    ['KDF (BCrypt)', timeit_bcrypt.best],
]
make_table(results_data)
apply_theme('basic')

Tipo,Tiempo (s)
Texto plano,0.0003
MD5,0.0004
SHA1,0.0003
SHA256,0.0003
KDF (BCrypt),31.5044
